In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:



df_1 = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
df_2 = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
df_3=pd.read_csv("/kaggle/input/data-storm-4/Testing-data.csv")

### Fill missing value

In [4]:
null_counts = df_1.isnull().sum()
print(null_counts)

item_description    35928
transaction_date        0
invoice_id           6320
customer_id             0
shop_id                 0
item_price              0
quantity_sold           0
dtype: int64


In [5]:
df_1 = df_1.drop_duplicates()


In [6]:
df_1['item_description'] = df_1['item_description'].fillna("NAN SODA 1L")

### Split the date to features

In [7]:
# df_1['transaction_date'].isnull().sum()

In [8]:
dates = df_1['transaction_date']
Dates = []

# Extract the substring corresponding to the day component
for date in dates:
    day_string = date[8:10]
    
    # Convert the day string to an integer
    day = int(day_string)
    
    Dates.append(day)
    
    
df_1['date'] =  Dates

In [9]:
# df_1['date'].isnull().sum()

In [10]:
sl_holidays = [pd.to_datetime(d) for d in ['2021-10-19', '2021-10-20', '2021-11-04', '2022-11-18', 
                                           '2022-12-18', '2022-12-25']]

# assume that the transaction_date column is a string column
df_1['transaction_date'] = pd.to_datetime(df_1['transaction_date'])

# now you can use the .dt accessor on the transaction_date column
df_1['is_weekend'] = df_1['transaction_date'].dt.weekday.isin([5, 6])

df_1['is_holiday'] = df_1['transaction_date'].isin(sl_holidays)

# convert boolean values to integers
df_1['is_weekend'] = df_1['is_weekend'].astype(int)

df_1['is_holiday'] = df_1['is_holiday'].astype(int)

In [11]:
df_1.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0
1,GINGER BEER 1.5L,2021-10-17 00:00:00+00:00,371.0,IA25,SHOP112,220,2,17,1,0
2,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0
3,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0
4,STRAWBERRY MILK 180ML,2021-10-23 00:00:00+00:00,1310.0,7S00,SHOP112,210,5,23,1,0


In [12]:
shop_invoice_count = df_1.groupby('shop_id')['invoice_id'].count()
# shop_invoice_counts = df_1.groupby('shop_id')['invoice_id'].value_counts().reset_index(name='invoice_count')
# shop_invoice_counts

In [13]:
# unique_categories = df_1['item_description'].unique()
# num_categories = df_1['item_description'].nunique()
# print(unique_categories)
# print(num_categories)
shop_invoice_count= pd.DataFrame(shop_invoice_count)
shop_invoice_count = shop_invoice_count.rename(columns={'invoice_id': 'count_invoice'})
shop_invoice_count 

,count_invoice
shop_id,
SHOP001,1864
SHOP002,4456
SHOP003,3381
SHOP004,3175
SHOP005,3825
...,...
SHOP123,1904
SHOP124,4025
SHOP125,1639


### Spilit the Item_description 

In [14]:

# extract item_name and item_size from item_description
df_1[['item_name', 'item_size']] = df_1['item_description'].str.extract('([\w\s]+) (\d*\.?\d+)\w{0,2}$')

# convert item_size to liters
df_1['item_size'] = df_1['item_size'].astype(float)
df_1.loc[df_1['item_description'].str.contains('ML', na=False), 'item_size'] /= 1000

# display the result
# df_1.head

### Merge two data set

In [15]:
# df_2 = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')

# Merge the two DataFrames on the primary key column
df_11 = pd.merge(shop_invoice_count, df_2, on='shop_id')
# df_1=pd.merge(df_1,shop_invoice_count, on='shop_id')
df_12= pd.merge(df_1, df_11, on='shop_id')
# df_12
df_1=df_12
df_1

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday,item_name,item_size,count_invoice,shop_area_sq_ft,shop_profile
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0,ORANGE BARLEY,1.5,244,678,Moderate
1,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0,TONIC PET,0.5,244,678,Moderate
2,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0,CREAM SODA,1.0,244,678,Moderate
3,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate
4,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,440,1,10,0,0,GINGER BEER,1.5,244,678,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442758,NAN SODA 1L,2021-12-09 00:00:00+00:00,7030409.0,QVA7,SHOP072,70,1,9,0,0,NAN SODA,1.0,4096,617,High
442759,FIT O ORANGE 200ML,2021-12-11 00:00:00+00:00,7030809.0,HXMG,SHOP072,140,2,11,1,0,FIT O ORANGE,0.2,4096,617,High
442760,BOTTLED DRINKING WATER 500ML,2021-12-11 00:00:00+00:00,7030820.0,OUH2,SHOP072,35,1,11,1,0,BOTTLED DRINKING WATER,0.5,4096,617,High
442761,GINGER BEER 1.5L,2021-10-21 00:00:00+00:00,NaN,5IJM,SHOP072,220,1,21,0,0,GINGER BEER,1.5,4096,617,High


In [16]:
# # df_1['item_price'] = df_1.groupby(['item_description'])['item_price'].apply(lambda x: x.fill(x.mode()[0]))
# df_1['item_price'] = df_1.groupby('item_description')['item_price'].transform(lambda x: x.fill(x.mode()[0]))

# df_1

def get_mode(series):
    return series.mode().iloc[0]

# Group the data frame by the primary key column and apply the mode function to each group
mode_values = df_1.groupby('item_description')['item_price'].transform(get_mode)

# Identify values that are not equal to the mode value within each group
not_mode = df_1['item_price'] != mode_values

# Replace those values with the mode value using the loc method
df_1.loc[not_mode, 'item_price'] = mode_values
df_1

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday,item_name,item_size,count_invoice,shop_area_sq_ft,shop_profile
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0,ORANGE BARLEY,1.5,244,678,Moderate
1,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0,TONIC PET,0.5,244,678,Moderate
2,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0,CREAM SODA,1.0,244,678,Moderate
3,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate
4,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442758,NAN SODA 1L,2021-12-09 00:00:00+00:00,7030409.0,QVA7,SHOP072,70,1,9,0,0,NAN SODA,1.0,4096,617,High
442759,FIT O ORANGE 200ML,2021-12-11 00:00:00+00:00,7030809.0,HXMG,SHOP072,60,2,11,1,0,FIT O ORANGE,0.2,4096,617,High
442760,BOTTLED DRINKING WATER 500ML,2021-12-11 00:00:00+00:00,7030820.0,OUH2,SHOP072,35,1,11,1,0,BOTTLED DRINKING WATER,0.5,4096,617,High
442761,GINGER BEER 1.5L,2021-10-21 00:00:00+00:00,NaN,5IJM,SHOP072,220,1,21,0,0,GINGER BEER,1.5,4096,617,High


### Add new feature

In [17]:
df_1['total_amount'] = df_1['item_price']*df_1['quantity_sold']
df_1

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday,item_name,item_size,count_invoice,shop_area_sq_ft,shop_profile,total_amount
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0,ORANGE BARLEY,1.5,244,678,Moderate,440
1,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0,TONIC PET,0.5,244,678,Moderate,320
2,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0,CREAM SODA,1.0,244,678,Moderate,300
3,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate,220
4,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442758,NAN SODA 1L,2021-12-09 00:00:00+00:00,7030409.0,QVA7,SHOP072,70,1,9,0,0,NAN SODA,1.0,4096,617,High,70
442759,FIT O ORANGE 200ML,2021-12-11 00:00:00+00:00,7030809.0,HXMG,SHOP072,60,2,11,1,0,FIT O ORANGE,0.2,4096,617,High,120
442760,BOTTLED DRINKING WATER 500ML,2021-12-11 00:00:00+00:00,7030820.0,OUH2,SHOP072,35,1,11,1,0,BOTTLED DRINKING WATER,0.5,4096,617,High,35
442761,GINGER BEER 1.5L,2021-10-21 00:00:00+00:00,NaN,5IJM,SHOP072,220,1,21,0,0,GINGER BEER,1.5,4096,617,High,220


### Filling the missing value

In [18]:
# df_1['item_name'] = df_1['item_name'].fillna("SODA")
mean_int = df_1['item_size'].mean().astype(int)
df_1['item_size'] =df_1['item_size'].fillna(mean_int)

df_1

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold,date,is_weekend,is_holiday,item_name,item_size,count_invoice,shop_area_sq_ft,shop_profile,total_amount
0,ORANGE BARLEY 1.5L,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,220,2,11,1,0,ORANGE BARLEY,1.5,244,678,Moderate,440
1,TONIC PET 500ML,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,160,2,13,0,0,TONIC PET,0.5,244,678,Moderate,320
2,CREAM SODA 1L,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,150,2,13,0,0,CREAM SODA,1.0,244,678,Moderate,300
3,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate,220
4,GINGER BEER 1.5L,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,220,1,10,0,0,GINGER BEER,1.5,244,678,Moderate,220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442758,NAN SODA 1L,2021-12-09 00:00:00+00:00,7030409.0,QVA7,SHOP072,70,1,9,0,0,NAN SODA,1.0,4096,617,High,70
442759,FIT O ORANGE 200ML,2021-12-11 00:00:00+00:00,7030809.0,HXMG,SHOP072,60,2,11,1,0,FIT O ORANGE,0.2,4096,617,High,120
442760,BOTTLED DRINKING WATER 500ML,2021-12-11 00:00:00+00:00,7030820.0,OUH2,SHOP072,35,1,11,1,0,BOTTLED DRINKING WATER,0.5,4096,617,High,35
442761,GINGER BEER 1.5L,2021-10-21 00:00:00+00:00,NaN,5IJM,SHOP072,220,1,21,0,0,GINGER BEER,1.5,4096,617,High,220


### Change the data frame Order

In [19]:
new_order = ['transaction_date','invoice_id','customer_id','shop_id','item_name','item_size','item_price','quantity_sold', 'shop_area_sq_ft','date','is_weekend','is_holiday', 'shop_profile','count_invoice']

# Reorder the columns in the dataframe
df_1 = df_1[new_order]
# df_1.head()
df_1

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice
0,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,ORANGE BARLEY,1.5,220,2,678,11,1,0,Moderate,244
1,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,TONIC PET,0.5,160,2,678,13,0,0,Moderate,244
2,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,CREAM SODA,1.0,150,2,678,13,0,0,Moderate,244
3,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,GINGER BEER,1.5,220,1,678,10,0,0,Moderate,244
4,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,GINGER BEER,1.5,220,1,678,10,0,0,Moderate,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442758,2021-12-09 00:00:00+00:00,7030409.0,QVA7,SHOP072,NAN SODA,1.0,70,1,617,9,0,0,High,4096
442759,2021-12-11 00:00:00+00:00,7030809.0,HXMG,SHOP072,FIT O ORANGE,0.2,60,2,617,11,1,0,High,4096
442760,2021-12-11 00:00:00+00:00,7030820.0,OUH2,SHOP072,BOTTLED DRINKING WATER,0.5,35,1,617,11,1,0,High,4096
442761,2021-10-21 00:00:00+00:00,NaN,5IJM,SHOP072,GINGER BEER,1.5,220,1,617,21,0,0,High,4096


### Feature Scaling

In [20]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Sample dataset
for i in df_1:
    if i not in ['transaction_date','invoice_id','customer_id','shop_id','item_name','shop_profile','item_encoded','is_weekend','is_holiday']:
        col_data = df_1[i]

    # Perform min-max scaling on the selected column
        scaled_data = (col_data - col_data.min()) / (col_data.max() - col_data.min())

        # Update the selected column in the original data frame with the scaled data
        df_1[i] = scaled_data



### Standardization

In [21]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# # create sample DataFrame
# # define standardization function
# scaler = StandardScaler()

# # fit and transform the DataFrame
# # df_1 = scaler.fit_transform(df_1)

# for i in df_1:
# # fit and transform column
#     if i not in ['transaction_date','invoice_id','customer_id','shop_id','item_name','shop_profile','item_encoded','is_weekend','is_holiday']:
#         df_1[i] = scaler.fit_transform(df_1[[i]])


# # convert the result back to a DataFrame
# # df_1 = pd.DataFrame(df_1, columns=df_1.columns)

In [22]:
df_1 = df_1.dropna(subset=['item_name'])


In [23]:
df_1.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice
0,2021-12-11 00:00:00+00:00,147.0,BGXA,SHOP008,ORANGE BARLEY,1.000000,0.300813,0.029412,0.487805,0.333333,1,0,Moderate,0.0
1,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,TONIC PET,0.242424,0.203252,0.029412,0.487805,0.400000,0,0,Moderate,0.0
2,2021-12-13 00:00:00+00:00,484.0,VN7V,SHOP008,CREAM SODA,0.621212,0.186992,0.029412,0.487805,0.400000,0,0,Moderate,0.0
3,2021-12-10 00:00:00+00:00,1000053.0,VT9C,SHOP008,GINGER BEER,1.000000,0.300813,0.019608,0.487805,0.300000,0,0,Moderate,0.0
4,2021-12-10 00:00:00+00:00,1000057.0,8QLS,SHOP008,GINGER BEER,1.000000,0.300813,0.019608,0.487805,0.300000,0,0,Moderate,0.0


In [24]:
# unique_categories = df_1['item_name'].unique()
# num_categories = df_1['item_name'].nunique()
# print(unique_categories)
# print(num_categories)

In [25]:
# null_counts = df_1.isnull().sum()
# print(null_counts)

### Sorting

In [26]:
df_sorted = df_1.sort_values(by='shop_id')
df_1=df_sorted
df_1.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.300813,0.019608,0.362003,0.500000,1,0,Low,0.277302
50814,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.029412,0.362003,0.333333,1,0,Low,0.277302
50815,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.019608,0.362003,0.333333,1,0,Low,0.277302
50816,2021-10-18 00:00:00+00:00,NaN,IQM6,SHOP001,GINGER BEER SUGAR FREE,0.242424,0.073171,0.019608,0.362003,0.566667,0,0,Low,0.277302
50817,2021-12-09 00:00:00+00:00,NaN,YI1R,SHOP001,GINGER BEER,1.000000,0.300813,0.029412,0.362003,0.266667,0,0,Low,0.277302


In [27]:
# df_sorted = df_sorted.dropna(subset=['item_name'])

### One hot encoding

In [28]:
# column_to_encode = 'item_name'
# encoded_df=df_1.copy()

# # Perform one-hot encoding on the selected column
# one_hot_encoded = pd.get_dummies(encoded_df[column_to_encode], prefix=column_to_encode)

# # Add the one-hot encoded columns to the original DataFrame
# encoded_df = pd.concat([encoded_df, one_hot_encoded], axis=1)

# # Drop the original column that was one-hot encoded
# encoded_df = encoded_df.drop(column_to_encode, axis=1)

# # Save the updated DataFrame to a new CSV file
# # df.to_csv('output.csv', index=False)
# encoded_df.head()

In [29]:
# encoded_df = encoded_df.dropna(subset=['item_name'])
# data = data.dropna()

In [30]:
# my_list = ['ORANGE BARLEY', 'TONIC PET', 'CREAM SODA', 'GINGER BEER','CREAM SODA APPLE POP', 'SODA', 'BOTTLED DRINKING WATER', 'LEMONADE','KIK COLA', 'STRAWBERRY MILK', 'SODA PET', 'FIT O MIXED FRUIT','NECTO', 'FIT O ORANGE', 'FIT O MANGO', 'GINGER BEER SUGAR FREE','ORANGE CRUSH', 'CHOCOLATE MILK', 'TWISTEE APPLE', 'DRY GINGER ALE PET', 'LIME CRUSH JUICE']
# len(my_list)
# my_dict = {i: my_list[i] for i in range(len(my_list))}

# # print(my_dict)
# swapped_dict = {v: k for k, v in my_dict.items()}

# print(swapped_dict)



### Instead (Label)

In [31]:
# # ['ORANGE BARLEY' 'TONIC PET' 'CREAM SODA' 'GINGER BEER'
# #  'CREAM SODA APPLE POP' 'SODA' 'BOTTLED DRINKING WATER' 'LEMONADE'
# #  'KIK COLA' 'STRAWBERRY MILK' 'SODA PET' 'FIT O MIXED FRUIT''NECTO'
# #  'FIT O ORANGE' 'FIT O MANGO' 'GINGER BEER SUGAR FREE' 'ORANGE CRUSH'
# #  'CHOCOLATE MILK' 'TWISTEE APPLE' 'DRY GINGER ALE PET' 'LIME CRUSH JUICE']

# # import pandas as pd
# from sklearn.preprocessing import LabelEncoder

# # Load the CSV file into a pandas DataFrame
# # df = pd.read_csv('filename.csv')

# # Create a LabelEncoder object
# le = LabelEncoder()

# # Fit the LabelEncoder on the column and transform the column
# #filtered_df['column_name_encoded'] = le.fit_transform(filtered_df["shop_profile"])

# # Save the updated DataFrame to a new CSV file
# # filtered_df.to_csv('filename_encoded.csv', index=False)

# # define the mapping between original and encoded values
# mapping = swapped_dict

# # fit the encoder to the 'shopprofile' column using the mapping
# le.fit([mapping[val] for val in df_1['item_name']])

# # transform the 'shopprofile' column using the fitted encoder
# df_1['item_encoded'] = le.transform([mapping[val] for val in df_1['item_name']])


# df_1.head()


import pandas as pd
from sklearn.preprocessing import LabelEncoder


# create a sample DataFrame

# create a LabelEncoder object
le = LabelEncoder()

# fit and transform the 'fruits' column
df_1['item_encoded'] = le.fit_transform(df_1['item_name'])
df_1['customer_id_encoded']=le.fit_transform(df_1['customer_id'])
# df_1['shop_encoded'] = le.fit_transform(df_1['shop_id'])
encoded_df=df_1

In [32]:
# df_1.head()

In [33]:
# grouped_df=df_1.copy()
# grouped_df['total']  = grouped_df.groupby(['item_name'])['quantity_sold'].transform('sum')
# grouped_df.head()

In [34]:
# grouped_df[["item_name","total"]]

In [35]:
# # Load the dataset into a pandas DataFrame

# # Create a line plot using seaborn
# plt.figure(figsize=(14,6))
# sns.lineplot(x='item_name', y='total', data=grouped_df)
# plt.xticks(rotation=90)

# # Add axis labels and a title to th

# # Show the plot
# plt.show()

In [36]:
encoded_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice,item_encoded,customer_id_encoded
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.300813,0.019608,0.362003,0.500000,1,0,Low,0.277302,15,50606
50814,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.029412,0.362003,0.333333,1,0,Low,0.277302,8,85139
50815,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.019608,0.362003,0.333333,1,0,Low,0.277302,8,85139
50816,2021-10-18 00:00:00+00:00,NaN,IQM6,SHOP001,GINGER BEER SUGAR FREE,0.242424,0.073171,0.019608,0.362003,0.566667,0,0,Low,0.277302,9,99809
50817,2021-12-09 00:00:00+00:00,NaN,YI1R,SHOP001,GINGER BEER,1.000000,0.300813,0.029412,0.362003,0.266667,0,0,Low,0.277302,8,183538


In [37]:
filtered_df = encoded_df[encoded_df['shop_profile'].notnull()]
na_df=encoded_df[encoded_df['shop_profile'].isnull()]
# encoded_df.head()
# filtered_df["shop_profile"]
# na_df.head()



### Label Encoding

In [38]:
# import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the CSV file into a pandas DataFrame
# df = pd.read_csv('filename.csv')

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder on the column and transform the column
#filtered_df['column_name_encoded'] = le.fit_transform(filtered_df["shop_profile"])

# Save the updated DataFrame to a new CSV file
# filtered_df.to_csv('filename_encoded.csv', index=False)

# define the mapping between original and encoded values
mapping = {'Moderate': 1, 'Low': 0, 'High': 2}

# fit the encoder to the 'shopprofile' column using the mapping
le.fit([mapping[val] for val in filtered_df['shop_profile']])

# transform the 'shopprofile' column using the fitted encoder
filtered_df['shop_profile_encoded'] = le.transform([mapping[val] for val in filtered_df['shop_profile']])


filtered_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice,item_encoded,customer_id_encoded,shop_profile_encoded
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.300813,0.019608,0.362003,0.500000,1,0,Low,0.277302,15,50606,0
50814,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.029412,0.362003,0.333333,1,0,Low,0.277302,8,85139,0
50815,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.019608,0.362003,0.333333,1,0,Low,0.277302,8,85139,0
50816,2021-10-18 00:00:00+00:00,NaN,IQM6,SHOP001,GINGER BEER SUGAR FREE,0.242424,0.073171,0.019608,0.362003,0.566667,0,0,Low,0.277302,9,99809,0
50817,2021-12-09 00:00:00+00:00,NaN,YI1R,SHOP001,GINGER BEER,1.000000,0.300813,0.029412,0.362003,0.266667,0,0,Low,0.277302,8,183538,0


In [39]:
# df_sorted = filtered_df.sort_values(by='shop_id')
# df_sorted

In [40]:
# calculate the index for splitting the dataframe
split_idx = int(0.8 * len(filtered_df))
# split the dataframe into training and testing sets
train_df =filtered_df.iloc[:split_idx]
test_df = filtered_df.iloc[split_idx:]
train_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice,item_encoded,customer_id_encoded,shop_profile_encoded
50020,2021-10-16 00:00:00+00:00,71340.0,9HZ9,SHOP001,ORANGE BARLEY,1.000000,0.300813,0.019608,0.362003,0.500000,1,0,Low,0.277302,15,50606,0
50814,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.029412,0.362003,0.333333,1,0,Low,0.277302,8,85139,0
50815,2021-12-11 00:00:00+00:00,3035533.0,G0AJ,SHOP001,GINGER BEER,0.621212,0.186992,0.019608,0.362003,0.333333,1,0,Low,0.277302,8,85139,0
50816,2021-10-18 00:00:00+00:00,NaN,IQM6,SHOP001,GINGER BEER SUGAR FREE,0.242424,0.073171,0.019608,0.362003,0.566667,0,0,Low,0.277302,9,99809,0
50817,2021-12-09 00:00:00+00:00,NaN,YI1R,SHOP001,GINGER BEER,1.000000,0.300813,0.029412,0.362003,0.266667,0,0,Low,0.277302,8,183538,0


In [41]:
X_train=train_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
y_train=train_df["shop_profile_encoded"]
X_test=test_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
y_test=test_df["shop_profile_encoded"]

In [42]:
# X_train.isnull().sum()

In [43]:
# X=filtered_df.drop(columns=['is_holiday','transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
# y=filtered_df["shop_profile_encoded"]

### Outliers

In [44]:
# def count_outliers(col_data):
#     Q1 = col_data.quantile(0.25)
#     Q3 = col_data.quantile(0.75)

#     # Calculate the interquartile range (IQR)
#     IQR = Q3 - Q1

#     # Define the threshold for outliers (1.5 times the IQR)
#     threshold = 1.5 * IQR

#     # Identify the outliers
#     outliers = col_data[(col_data < Q1 - threshold) | (col_data > Q3 + threshold)]

#     # Count the number of outliers
#     num_outliers = len(outliers)

#     print("Number of outliers in column", ":", num_outliers)



In [45]:
# X = X.fillna(0)
# X.head()

In [46]:
# from sklearn.model_selection import train_test_split

# y = filtered_df['shop_profile_encoded']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=50)

In [47]:
# X_train.head(5)

### Logistic

In [48]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# load the data into a DataFrame
# data = pd.read_csv('data.csv')

# # split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(data[['feature1', 'feature2', 'feature3']], data['target'], test_size=0.2)

# create a logistic regression object and fit the training data
lr = LogisticRegression()
lr.fit(X_train, y_train)

# evaluate the performance of the model on the test set
score = lr.score(X_test, y_test)
print(score * 100)

31.140557138510054


### SVM

In [49]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC


# # Load the iris datase

# # Define the SVM classifier
# svm = SVC(kernel='linear')

# # Fit the training data
# svm.fit(X_train, y_train)

# # Predict the classes of new data
# # y_pred = clf.predict(X_test)

# # Print the accuracy score
# # print("Accuracy:", clf.score(X_test, y_test))


In [50]:
# import pandas as pd
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # load the data into a DataFrame
# # data = pd.read_csv('data.csv')

# # # split the data into training and test sets
# # X_train, X_test, y_train, y_test = train_test_split(data[['feature1', 'feature2', 'feature3']], data['target'], test_size=0.2)

# # create an SVM object and fit the training data
# svm = SVC(kernel='linear')
# svm.fit(X_train, y_train)

# # predict the class labels for the test set
# y_pred = svm.predict(X_test)

# # evaluate the performance of the model on the test set
# score = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(score * 100))


In [51]:
# import xgboost as xgb
# import numpy as np

# # Convert data to DMatrix format
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# # Set hyperparameters
# params = {
#     'objective': 'multi:softmax',
#     'num_class': len(np.unique(y_train)),
#     'max_depth': 6,
#     'eta': 0.3,
#     'eval_metric': 'merror'
# }

# # Train the model
# model = xgb.train(params, dtrain, num_boost_round=10)

# # Predict on test set
# y_pred = model.predict(dtest)

# # Evaluate the model
# accuracy = np.sum(y_pred == y_test) / len(y_test)
# print('Accuracy:', accuracy)

### Neural Network

In [52]:
# from sklearn.neural_network import MLPClassifier

# # create neural network classifier object
# clf = MLPClassifier(hidden_layer_sizes=(10,5), activation='relu', solver='adam', max_iter=200)

# # fit the classifier to the training data
# clf.fit(X_train, y_train)

# # predict the class labels for the test data
# y_pred = clf.predict(X_test)

# # calculate the accuracy of the classifier on the test data
# accuracy = clf.score(X_test, y_test)*100
# print('Accuracy of the  MLPClassifier is equal ' + str(round(accuracy, 2)) + ' %.')

### Random forest

In [53]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100, random_state=50)

# Train the random forest classifier on the training data
rf.fit(X_train, y_train)

# Predict the classes of the testing data
y_pred = rf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = rf.score(X_test, y_test)*100
print('Accuracy of the  RandomForestClassifier is equal ' + str(round(accuracy, 2)) + ' %.')

Accuracy of the  RandomForestClassifier is equal 47.01 %.


### Using DecisionTreeClassifier

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Load the dataset into a pandas dataframe
# df = pd.read_csv('dataset.csv')

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3)

# Create a decision tree classifier object
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 0.46553154997371987


### Naive_bayes algorithm

In [55]:
# # Import required libraries
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer


# # Split data into training and testing sets

# # Train the model
# clf = MultinomialNB()
# clf.fit(X_train, y_train)

# # Predict using the trained model
# y_pred = clf.predict(X_test)

# # Evaluate the model accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

### Using KNN algorithm

In [56]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# import pandas as pd


# knn = KNeighborsClassifier(n_neighbors=5)

# # Fit the classifier to the training data
# knn.fit(X_train, y_train)

# # Predict the classes of the test data
# y_pred = knn.predict(X_test)

# # Evaluate the accuracy of the classifier
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

### Gradient Boosting.

In [57]:
# # Import required libraries
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# # Load your dataset
# # Assuming X is your feature matrix and y is your target vector

# # Split data into training and testing sets

# # Train the model
# clf = GradientBoostingClassifier()
# clf.fit(X_train, y_train)

# # Predict using the trained model
# y_pred = clf.predict(X_test)

# # Evaluate the model accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [58]:
test_df = na_df
X_train=filtered_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','shop_profile_encoded','item_name'])
y_train=filtered_df["shop_profile_encoded"]
X_test=test_df.drop(columns=['transaction_date','invoice_id','customer_id','shop_id','shop_profile','item_name'])

In [59]:
test_df.head()

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice,item_encoded,customer_id_encoded
433255,2021-11-29 00:00:00+00:00,982435.0,SS49,SHOP002,NAN SODA,0.621212,0.056911,0.049020,0.435173,0.933333,0,0,NaN,0.720986,13,153385
433258,2021-12-10 00:00:00+00:00,984928.0,130R,SHOP002,NAN SODA,0.621212,0.056911,0.039216,0.435173,0.300000,0,0,NaN,0.720986,13,5671
433257,2021-12-10 00:00:00+00:00,984862.0,407B,SHOP002,GINGER BEER,1.000000,0.300813,0.019608,0.435173,0.300000,0,0,NaN,0.720986,8,21185
433254,2021-11-28 00:00:00+00:00,982168.0,288E,SHOP002,CREAM SODA,1.000000,0.300813,0.019608,0.435173,0.900000,1,0,NaN,0.720986,2,11717
433256,2021-12-02 00:00:00+00:00,983045.0,ZWGI,SHOP002,GINGER BEER,1.000000,0.300813,0.019608,0.435173,0.033333,0,0,NaN,0.720986,8,191102


In [60]:
print(test_df.shape[0])
print(X_test.shape[0])

81276
81276


In [61]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [62]:
# from sklearn.ensemble import RandomForestClassifier

# # Create a random forest classifier with 100 trees
# rf = RandomForestClassifier(n_estimators=100, random_state=50)

# # Train the random forest classifier on the training data
# rf.fit(X_train, y_train)

# # Predict the classes of the testing data
# y_pred = rf.predict(X_test)

# # Evaluate the accuracy of the mode

In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Load the dataset into a pandas dataframe
# df = pd.read_csv('dataset.csv')

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3)

# Create a decision tree classifier object
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

In [64]:
# Convert the numpy array to a dataframe with column name "y_pred"
df_4 = pd.DataFrame({'y_pred': y_pred})
df_4

,y_pred
0,1
1,1
2,1
3,1
4,1
...,...
81271,2
81272,2
81273,2
81274,2


In [65]:
test_df = test_df.reset_index(drop=True)
test_df

,transaction_date,invoice_id,customer_id,shop_id,item_name,item_size,item_price,quantity_sold,shop_area_sq_ft,date,is_weekend,is_holiday,shop_profile,count_invoice,item_encoded,customer_id_encoded
0,2021-11-29 00:00:00+00:00,982435.0,SS49,SHOP002,NAN SODA,0.621212,0.056911,0.049020,0.435173,0.933333,0,0,NaN,0.720986,13,153385
1,2021-12-10 00:00:00+00:00,984928.0,130R,SHOP002,NAN SODA,0.621212,0.056911,0.039216,0.435173,0.300000,0,0,NaN,0.720986,13,5671
2,2021-12-10 00:00:00+00:00,984862.0,407B,SHOP002,GINGER BEER,1.000000,0.300813,0.019608,0.435173,0.300000,0,0,NaN,0.720986,8,21185
3,2021-11-28 00:00:00+00:00,982168.0,288E,SHOP002,CREAM SODA,1.000000,0.300813,0.019608,0.435173,0.900000,1,0,NaN,0.720986,2,11717
4,2021-12-02 00:00:00+00:00,983045.0,ZWGI,SHOP002,GINGER BEER,1.000000,0.300813,0.019608,0.435173,0.033333,0,0,NaN,0.720986,8,191102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81271,2021-12-08 00:00:00+00:00,4010536.0,U4F9,SHOP114,NECTO,1.000000,0.300813,0.019608,0.275995,0.233333,0,0,NaN,0.638480,14,160504
81272,2021-12-08 00:00:00+00:00,4010536.0,U4F9,SHOP114,GINGER BEER,1.000000,0.300813,0.029412,0.275995,0.233333,0,0,NaN,0.638480,8,160504
81273,2021-12-08 00:00:00+00:00,4010536.0,U4F9,SHOP114,LEMONADE,1.000000,0.300813,0.029412,0.275995,0.233333,0,0,NaN,0.638480,11,160504
81274,2021-12-11 00:00:00+00:00,4011401.0,Z5W1,SHOP114,NAN SODA,0.621212,0.056911,0.029412,0.275995,0.333333,1,0,NaN,0.638480,13,187151


In [66]:
result_df = pd.concat([test_df, df_4], axis=1)

In [67]:
new_df =result_df.loc[:, ["shop_id", "y_pred"]]
new_df

,shop_id,y_pred
0,SHOP002,1
1,SHOP002,1
2,SHOP002,1
3,SHOP002,1
4,SHOP002,1
...,...,...
81271,SHOP114,2
81272,SHOP114,2
81273,SHOP114,2
81274,SHOP114,2


In [68]:
mode_df = new_df.groupby('shop_id')['y_pred'].apply(lambda x: x.mean()).reset_index()

mode_df['y_pred'] = mode_df['y_pred'].astype(int)

In [69]:
mode_df=mode_df.replace([0,1,2],["Low","Moderate","High"])

In [70]:
mode_df = mode_df.rename(columns={'y_pred': 'shop_profile'})

In [71]:
df_3.head()

,shop_id,shop_profile
0,SHOP046,NaN
1,SHOP024,NaN
2,SHOP023,NaN
3,SHOP097,NaN
4,SHOP044,NaN


In [72]:
# df_3=pd.read_csv("/kaggle/input/data-storm-4/Testing-data.csv")
df_3.drop("shop_profile", axis =1, inplace = True)
merged_df = pd.merge(df_3, mode_df, on='shop_id')

In [73]:
merged_df.head()

,shop_id,shop_profile
0,SHOP046,Low
1,SHOP024,Moderate
2,SHOP023,Low
3,SHOP097,Moderate
4,SHOP044,Moderate


In [74]:
merged_df.to_csv("Day_2_Submission2.csv",index=False)